In [1]:
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanizehttp://localhost:8888/notebooks/live_edit_music_gen.ipynb#
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

In [2]:
# from google.colab import drive
# drive.mount('/gdrive') 
# %cd /gdrive

In [3]:
pip install mido

Note: you may need to restart the kernel to use updated packages.


In [4]:
from mido import MidiFile, MidiTrack, Message
import numpy as np

num_notes = 118
samples_per_measure = 118

def midi_to_samples(fname):
    print(fname)
    has_time_sig = False
    flag_warning = False
    mid = MidiFile(fname)
    ticks_per_beat = mid.ticks_per_beat
    ticks_per_measure = 4 * ticks_per_beat

    for i, track in enumerate(mid.tracks):
        for msg in track:
            if msg.type == 'time_signature':
                new_tpm = msg.numerator * ticks_per_beat * 4 / msg.denominator
                if has_time_sig and new_tpm != ticks_per_measure:
                    flag_warning = True
                ticks_per_measure = new_tpm
                has_time_sig = True
    if flag_warning:
        print("  ^^^^^^ WARNING ^^^^^^")
        print("    " + fname)
        print("    Detected multiple distinct time signatures.")
        print("  ^^^^^^ WARNING ^^^^^^")
        return []

    all_notes = {}
    for i, track in enumerate(mid.tracks):
        abs_time = 0
        for msg in track:
            abs_time += msg.time
            if msg.type == 'note_on':
                if msg.velocity == 0:
                    continue
            # print("Msg : ",msg)
                note = msg.note - (128 - num_notes)/2
            # print(note)
                assert(note >= 0 and note < num_notes)
                if note not in all_notes:
                    all_notes[note] = []
                else:
                    single_note = all_notes[note][-1]
                    if len(single_note) == 1:
                        single_note.append(single_note[0] + 1)
                all_notes[note].append([abs_time * samples_per_measure / ticks_per_measure])
            elif msg.type == 'note_off':
                note = msg.note - (128 - num_notes)/2
                if note not in all_notes:
                    continue
                if len(all_notes[note][-1]) != 1:
                    continue
                all_notes[note][-1].append(abs_time * samples_per_measure / ticks_per_measure)
    for note in all_notes:
        for start_end in all_notes[note]:
            if len(start_end) == 1:
                start_end.append(start_end[0] + 1)
    samples = []
    for note in all_notes:
        for start, end in all_notes[note]:
            note=int(note)
      # print(note)
            sample_ix = int(start / samples_per_measure)
            while len(samples) <= sample_ix:
                samples.append(np.zeros((samples_per_measure, num_notes), dtype=np.uint8))
      # print(samples)
#             sample = samples[sample_ix]
      # print(sample_ix)
            start_ix = start - sample_ix * samples_per_measure
      # print(start_ix)
            if False:
                end_ix = min(end - sample_ix * samples_per_measure, samples_per_measure)
                while start_ix < end_ix:
                    samples[sample_ix][int(start_ix)][note] = 1
                    start_ix += 1
            else:
                samples[sample_ix][int(start_ix)][note] = 1
    return samples

def samples_to_midi(samples, fname, ticks_per_sample, thresh=0.5):
	mid = MidiFile()
	track = MidiTrack()
	mid.tracks.append(track)
	ticks_per_beat = mid.ticks_per_beat
	ticks_per_measure = 4 * ticks_per_beat
	ticks_per_sample = int(ticks_per_measure / samples_per_measure)
	abs_time = 0
	last_time = 0
	for sample in samples:
		for y in range(sample.shape[0]):
			abs_time += ticks_per_sample
			for x in range(sample.shape[1]):
				note = int(x + (128 - num_notes)/2)
				if sample[y,x] >= thresh and (y == 0 or sample[y-1,x] < thresh):
					delta_time = abs_time - last_time
					track.append(Message('note_on', note=note, velocity=127, time=delta_time))
					last_time = abs_time
				if sample[y,x] >= thresh and (y == sample.shape[0]-1 or sample[y+1,x] < thresh):
					delta_time = abs_time - last_time
					track.append(Message('note_off', note=note, velocity=127, time=delta_time))
					last_time = abs_time
	mid.save("Songs/"+fname)

In [5]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install wave

Note: you may need to restart the kernel to use updated packages.


In [7]:
# !apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

In [8]:
pip install pyaudio

Note: you may need to restart the kernel to use updated packages.


In [9]:
# import os
# os.environ['SDL_VIDEODRIVER']='dummy'

In [10]:
import pygame
import random, math
import numpy as np
import pyaudio
import os

#User constants
if not os.path.exists('Songs'):
    os.mkdir("Songs")
device = "cpu"
dir_name = 'History/'
sub_dir_name = 'e2000/'
sample_rate = 48000
note_dt = 2000        #Num Samples
note_duration = 20000 #Num Samples
note_decay = 5.0 / sample_rate
num_params = 120
num_measures = 16
num_sigmas = 5.0
note_thresh = 32
use_pca = True
is_ae = True

background_color = (255, 255, 255)
edge_color = (60, 60, 60)
slider_colors = [(90, 20, 20), (90, 90, 20), (20, 90, 20), (20, 90, 90), (20, 20, 90), (90, 20, 90)]

note_w = 118
note_h = 118
note_pad = 2

notes_rows = int(num_measures / 8)
notes_cols = 8

slider_num = min(40, num_params)
slider_h = 200
slider_pad = 5
tick_pad = 4

control_w = 210
control_h = 30
control_pad = 5
control_num = 3
control_colors = [(255,0,0), (0,255,0), (0,0,255)]
control_inits = [0.75, 0.5, 0.5]

#Derived constants
notes_w = notes_cols * (note_w + note_pad*2)
notes_h = notes_rows * (note_h + note_pad*2)
sliders_w = notes_w
sliders_h = slider_h + slider_pad*2
controls_w = control_w * control_num
controls_h = control_h
window_w = notes_w
window_h = int(notes_h + sliders_h + controls_h)
slider_w = int((window_w - slider_pad*2) / slider_num)
notes_x = 0
notes_y = sliders_h
sliders_x = slider_pad
sliders_y = slider_pad
controls_x = int((window_w - controls_w) / 2)
controls_y = notes_h + sliders_h

#Global variables
prev_mouse_pos = None
mouse_pressed = 0
cur_slider_ix = 0
cur_control_ix = 0
volume = 3000
instrument = 0
needs_update = True
cur_params = np.zeros((num_params,), dtype=np.float32)
cur_notes = np.zeros((num_measures, note_h, note_w), dtype=np.uint8)
cur_controls = np.array(control_inits, dtype=np.float32)

#Setup audio stream
audio = pyaudio.PyAudio()
audio_notes = []
audio_time = 0
note_time = 0
note_time_dt = 0
audio_reset = False
audio_pause = False
def audio_callback(in_data, frame_count, time_info, status):
    global audio_time
    global audio_notes
    global audio_reset
    global note_time
    global note_time_dt

    #Check if needs restart
    if audio_reset:
        audio_notes = []
        audio_time = 0
        note_time = 0
        note_time_dt = 0
        audio_reset = False

    #Check if paused
    if audio_pause and status is not None:
        data = np.zeros((frame_count,), dtype=np.float32)
        return (data.tobytes(), pyaudio.paContinue)

    #Find and add any notes in this time window
    cur_dt = note_dt
    while note_time_dt < audio_time + frame_count:
        measure_ix = int(note_time / note_h)#------------------------------------------------------------------------
        if measure_ix >= num_measures:
            break
        note_ix = note_time % note_h
        notes = np.where(cur_notes[int(measure_ix), note_ix] >= note_thresh)[0]
        for note in notes:
            freq = 2 * 38.89 * pow(2.0, note / 12.0) / sample_rate
            audio_notes.append((note_time_dt, freq))
        note_time += 1
        note_time_dt += cur_dt

    #Generate the tones
    data = np.zeros((frame_count,), dtype=np.float32)
    for t,f in audio_notes:
        x = np.arange(audio_time - t, audio_time + frame_count - t)
        x = np.maximum(x, 0)

        if instrument == 0:
            w = np.sign(1 - np.mod(x * f, 2))            #Square
        elif instrument == 1:
            w = np.mod(x * f - 1, 2) - 1                 #Sawtooth
        elif instrument == 2:
            w = 2*np.abs(np.mod(x * f - 0.5, 2) - 1) - 1 #Triangle
        elif instrument == 3:
            w = np.sin(x * f * math.pi)                  #Sine

        #w = np.floor(w*8)/8
        w[x == 0] = 0
        w *= volume * np.exp(-x*note_decay)
        data += w
    data = np.clip(data, -32000, 32000).astype(np.int16)

    #Remove notes that are too old
    audio_time += frame_count
    audio_notes = [(t,f) for t,f in audio_notes if audio_time < t + note_duration]

    #Reset if loop occurs
    if note_time / note_h >= num_measures:
        audio_time = 0
        note_time = 0
        note_time_dt = 0
        audio_notes = []

    #Return the sound clip
    return (data.tobytes(), pyaudio.paContinue)

#Keras
print("Loading Keras...")
import os
import keras
import uuid
print("Keras Version: " + keras.__version__)
from keras.models import Model, Sequential, load_model
# from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
# from keras.layers.convolutional import Conv2D, Conv2DTranspose, ZeroPadding2D
# from keras.layers.pooling import MaxPooling2D
# from keras.layers.noise import GaussianNoise
# from keras.layers.local import LocallyConnected2D
# from keras.optimizers import Adam, RMSprop, SGD
# from keras.regularizers import l2
# from keras.losses import binary_crossentropy
# from keras.layers.advanced_activations import ELU
# from keras.preprocessing.image import ImageDataGenerator
# from keras.utils import plot_model
from keras import backend as K
K.set_image_data_format('channels_first')

print("Loading Encoder...")
model = load_model(dir_name + 'model.h5')
enc = K.function([model.get_layer('encoder').input, K.learning_phase()],
                 [model.layers[-1].output])
enc_model = Model(inputs=model.input, outputs=model.get_layer('pre_encoder').output)

print("Loading Statistics...")
means = np.load(dir_name + sub_dir_name + 'means.npy')
evals = np.load(dir_name + sub_dir_name + 'evals.npy')
evecs = np.load(dir_name + sub_dir_name + 'evecs.npy')
stds = np.load(dir_name + sub_dir_name + 'stds.npy')

print("Loading Songs...")
y_samples = np.load('samples.npy')
y_lengths = np.load('lengths.npy')

#Open a window
pygame.init()
pygame.font.init()
screen = pygame.display.set_mode((window_w, window_h+150))
notes_surface = screen.subsurface((notes_x, notes_y, notes_w, notes_h))
pygame.display.set_caption('MusicEdit')
font = pygame.font.SysFont("monospace", 15)

#Start the audio stream
audio_stream = audio.open(
    format=audio.get_format_from_width(2),
    channels=1,
    rate=sample_rate,
    output=True,
    stream_callback=audio_callback)
audio_stream.start_stream()

def update_mouse_click(mouse_pos):
    global cur_slider_ix
    global cur_control_ix
    global mouse_pressed
    x = (mouse_pos[0] - sliders_x)
    y = (mouse_pos[1] - sliders_y)

    if x >= 0 and y >= 0 and x < sliders_w and y < sliders_h:
        cur_slider_ix = int(x / slider_w)
        mouse_pressed = 1

    x = (mouse_pos[0] - controls_x)
    y = (mouse_pos[1] - controls_y)
    if x >= 0 and y >= 0 and x < controls_w and y < controls_h:
        cur_control_ix = int(x / control_w)
        mouse_pressed = 2

def apply_controls():
    global note_thresh
    global note_dt
    global volume

    note_thresh = (1.0 - cur_controls[0]) * 200 + 10
    note_dt = (1.0 - cur_controls[1]) * 1800 + 200
    volume = cur_controls[2] * 6000

def update_mouse_move(mouse_pos):
    global needs_update

    if mouse_pressed == 1:
        y = (mouse_pos[1] - sliders_y)
        if y >= 0 and y <= slider_h:
            val = (float(y) / slider_h - 0.5) * (num_sigmas * 2)
            cur_params[cur_slider_ix] = val
            needs_update = True
    elif mouse_pressed == 2:
        x = (mouse_pos[0] - (controls_x + cur_control_ix*control_w))
        if x >= control_pad and x <= control_w - control_pad:
            val = float(x - control_pad) / (control_w - control_pad*2)
            cur_controls[cur_control_ix] = val
            apply_controls()

def draw_controls():
    x1=0
    y=0
    control_names = ['Number of Notes','Playback Speed', 'Volume']
    for i in range(control_num):
        x = controls_x + i * control_w + control_pad
        if i==0 :
            x1=x
        y = controls_y + control_pad
        w = control_w - control_pad*2
        h = control_h - control_pad*2
        col = control_colors[i]

        pygame.draw.rect(screen, col, (x, y, int(w*cur_controls[i]), h))
        pygame.draw.rect(screen, (0,0,0), (x, y, w, h), 1)
        text = font.render(control_names[i], True, (0,0,0))
        screen.blit(text, (x,y+20))
    green = (0, 255, 0)
    blue = (0, 0, 128)
#     inst = 'Press SPACE to pause'+ '\n'+ 'Press TAB to reset'+ '\n' +'Use 1,2,3,4 for different kinds of tones'+'\n'+'Press G to save the generated music '+'\n'+ 'Press C to randomly ajust the sliders to generate Music '
    arr = ['Press SPACE to pause','Press TAB to reset','Use 1,2,3,4 for different kinds of tones','Press G to save the generated music ','Press C to randomly ajust the sliders to generate Music ']
#     text = font.render(inst, True, (0,0,0))
#     screen.blit(text, (x1,y+20))
    j1=60
    for j in range(5):
        text = font.render(arr[j], True, (0,0,0))
        screen.blit(text, (x1,y+j1))
        j1+=20

def draw_sliders():
    for i in range(slider_num):
        slider_color = slider_colors[i % len(slider_colors)]
        x = sliders_x + i * slider_w
        y = sliders_y

        cx = int(x + slider_w / 2)
        cy_1 = y
        cy_2 = y + slider_h
        pygame.draw.line(screen, slider_color, (cx, cy_1), (cx, cy_2))
#         print(cy_2-cy_1)

        cx_1 = x + tick_pad
        cx_2 = x + slider_w - tick_pad
        for j in range(int(num_sigmas * 2 + 1)):
            ly = y + slider_h/2.0 + (j-num_sigmas)*slider_h/(num_sigmas*2.0)
            ly = int(ly)
            col = (0,0,0) if j - num_sigmas == 0 else slider_color
            pygame.draw.line(screen, col, (cx_1, ly), (cx_2, ly))

        py = int(y + int((cur_params[i] / (num_sigmas * 2) + 0.5) * slider_h))
        pygame.draw.circle(screen, slider_color, (cx, py), int((slider_w - tick_pad)/2))

def notes_to_img(notes):
    output = np.full((3, notes_h, notes_w), 64, dtype=np.uint8)

    for i in range(notes_rows):
        for j in range(notes_cols):
            x = note_pad + j*(note_w + note_pad*2)
            y = note_pad + i*(note_h + note_pad*2)
            ix = i*notes_cols + j

            measure = np.rot90(notes[ix])
            played_only = np.where(measure >= note_thresh, 255, 0)
            output[0,y:y+note_h,x:x+note_w] = np.minimum(measure * (255.0 / note_thresh), 255.0)
            output[1,y:y+note_h,x:x+note_w] = played_only
            output[2,y:y+note_h,x:x+note_w] = played_only

    return np.transpose(output, (2, 1, 0))

def draw_notes():
    pygame.surfarray.blit_array(notes_surface, notes_to_img(cur_notes))

    measure_ix = int(note_time / note_h)
    note_ix = note_time % note_h
    x = notes_x + note_pad + (measure_ix % notes_cols) * (note_w + note_pad*2) + note_ix
    y = notes_y + note_pad + int(measure_ix / notes_cols) * (note_h + note_pad*2)
    pygame.draw.rect(screen, (255,255,0), (x, y, 4, note_h), 0)

#Main loop
running = True
rand_ix = 0
cur_len = 0
apply_controls()
while running:
    #Process events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            break
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if pygame.mouse.get_pressed()[0]:
                prev_mouse_pos = pygame.mouse.get_pos()
                update_mouse_click(prev_mouse_pos)
                update_mouse_move(prev_mouse_pos)
            elif pygame.mouse.get_pressed()[2]:
                cur_params = np.zeros((num_params,), dtype=np.float32)
                needs_update = True
        elif event.type == pygame.MOUSEBUTTONUP:
            mouse_pressed = 0
            prev_mouse_pos = None
        elif event.type == pygame.MOUSEMOTION and mouse_pressed > 0:
            update_mouse_move(pygame.mouse.get_pos())
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                cur_params = np.clip(np.random.normal(0.0, 1.0, (num_params,)), -num_sigmas, num_sigmas)
                needs_update = True
                audio_reset = True
            if event.key == pygame.K_e:
                cur_params = np.clip(np.random.normal(0.0, 2.0, (num_params,)), -num_sigmas, num_sigmas)
                needs_update = True
                audio_reset = True
            if event.key == pygame.K_o:
                print("RandIx: " + str(rand_ix))
                if is_ae:
                    example_song = y_samples[cur_len:cur_len + num_measures]
                    cur_notes = example_song * 255
                    x = enc_model.predict(np.expand_dims(example_song, 0), batch_size=1)[0]
                    cur_len += y_lengths[rand_ix]
                    rand_ix += 1
                else:
                    rand_ix = np.array([rand_ix], dtype=np.int64)
                    x = enc_model.predict(rand_ix, batch_size=1)[0]
                    rand_ix = (rand_ix + 1) % model.layers[0].input_dim

                if use_pca:
                    cur_params = np.dot(x - means, evecs.T) / evals
                else:
                    cur_params = (x - means) / stds

                needs_update = True
                audio_reset = True
            if event.key == pygame.K_g:
                audio_pause = True
                audio_reset = True
                filename = uuid.uuid4().hex
                midi_filename = filename + '.mid'
                param_filename = 'Songs/' + filename + '.npy'
                samples_to_midi(cur_notes, midi_filename, 16, note_thresh)
#                 with open(param_filename,'wb') as f:
#                     np.save(f,cur_params)
#                     np.save(f,cur_notes)
#                     np.save(f,cur_controls)
#                 save_audio = ''
#                 while True:
#                     save_audio += audio_callback(None, 1024, None, None)[0]
#                     if audio_time == 0:
#                         break
#                 wave_output = wave.open('live.wav', 'w')
#                 wave_output.setparams((1, 2, sample_rate, 0, 'NONE', 'not compressed'))
#                 wave_output.writeframes(save_audio)
#                 wave_output.close()
                audio_pause = False
            if event.key == pygame.K_ESCAPE:
                running = False
                break
            if event.key == pygame.K_SPACE:
                audio_pause = not audio_pause
            if event.key == pygame.K_TAB:
                audio_reset = True
            if event.key == pygame.K_1:
                instrument = 0
            if event.key == pygame.K_2:
                instrument = 1
            if event.key == pygame.K_3:
                instrument = 2
            if event.key == pygame.K_4:
                instrument = 3
            if event.key == pygame.K_c:
                y = np.expand_dims(np.where(cur_notes > note_thresh, 1, 0), 0)
                x = enc_model.predict(y)[0]
                if use_pca:
                    cur_params = np.dot(x - means, evecs.T) / evals
                else:
                    cur_params = (x - means) / stds
                needs_update = True

    #Check if we need an update
    if needs_update:
        if use_pca:
            x = means + np.dot(cur_params * evals, evecs)
        else:
            x = means + stds * cur_params
        x = np.expand_dims(x, axis=0)
        y = enc([x, 0])[0][0]
        cur_notes = (y * 255.0).astype(np.uint8)
        needs_update = False

    #Draw to the screen
    screen.fill(background_color)
    draw_notes()
    draw_sliders()
    draw_controls()

    #Flip the screen buffer
    pygame.display.flip()
    pygame.time.wait(10)

#Close the audio stream
audio_stream.stop_stream()
audio_stream.close()
audio.terminate()
pygame.quit()
quit() 

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading Keras...


Using TensorFlow backend.


Keras Version: 2.3.1
Loading Encoder...
Loading Statistics...
Loading Songs...
RandIx: 0
RandIx: 1
RandIx: 2
RandIx: 3
RandIx: 4
RandIx: 5
RandIx: 6
RandIx: 7
RandIx: 8
RandIx: 9
RandIx: 10
RandIx: 11
RandIx: 12
RandIx: 13
RandIx: 14
RandIx: 15
RandIx: 16
RandIx: 17
RandIx: 18
RandIx: 19
RandIx: 20
RandIx: 21
RandIx: 22
RandIx: 23
RandIx: 24
RandIx: 25
RandIx: 26
RandIx: 27
RandIx: 28
RandIx: 29
RandIx: 30
RandIx: 31
RandIx: 32
RandIx: 33
RandIx: 34
RandIx: 35
RandIx: 36
RandIx: 37
RandIx: 38
RandIx: 39
RandIx: 40
RandIx: 41
RandIx: 42
RandIx: 43
RandIx: 44
RandIx: 45
RandIx: 46
RandIx: 47
RandIx: 48
RandIx: 49
RandIx: 50
RandIx: 51
RandIx: 52
RandIx: 53
RandIx: 54
RandIx: 55
RandIx: 56
RandIx: 57
RandIx: 58
RandIx: 59
RandIx: 60
RandIx: 61
RandIx: 62
RandIx: 63
RandIx: 64
RandIx: 65
RandIx: 66
RandIx: 67
RandIx: 68
RandIx: 69
RandIx: 70
RandIx: 71
RandIx: 72
RandIx: 73
RandIx: 74
RandIx: 75
RandIx: 76
RandIx: 77
RandIx: 78
RandIx: 79
RandIx: 80
RandIx: 81
RandIx: 82
RandIx: 83
RandIx:

In [11]:
import uuid

unique_filename = uuid.uuid4().hex

print(unique_filename)

69e5ae9173d44818a7d4ca134c32d66b


In [12]:
# import tkinter as tk
# from tkinter import filedialog as fd 
# import numpy as np

# def callback():
#     name= fd.askopenfilename() 
#     ex = np.load(name)
#     print(ex)
    
# errmsg = 'Error!'
# tk.Button(text='Click to Open File', 
#        command=callback).pack(fill=tk.X)
# tk.mainloop()

In [13]:
# import os
# os.mkdir("SOngs")

In [14]:
# from tkinter import Tk
# from tkinter.filedialog import askopenfilename

# Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
# filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
# print(filename)

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
